In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Apr  7 22:08:46 2019

@author: Deechean
"""
from cifar10 import cifar10
import os
import tensorflow as tf
from lenet5 import LeNet5_LERU
import tf_general as tfg
import numpy as np
import aws_boto3

FLAGS = tf.flags.FLAGS
# learn Rate = 0.01, keep Prob= 0.5, image augument = T

try:
    tf.flags.DEFINE_string('f', '', 'kernel')
    tf.flags.DEFINE_integer('epoch', 50000, 'epoch')
    tf.flags.DEFINE_integer('batch_size',200, 'batch size')
    tf.flags.DEFINE_integer('test_size', 200, 'test size')
    tf.flags.DEFINE_float('lr', 0.01, 'learning rate')
    tf.flags.DEFINE_float('keep_prob', 0.5, 'keep prob for drop lay')
    tf.flags.DEFINE_boolean('augument', True, 'if image augument is applied')
    
    tf.flags.DEFINE_float('ckpt_frequency', 250, 'frequency to save checkpoint')
    tf.flags.DEFINE_boolean('restore', False, 'restore from checkpoint and run test')
    print('parameters were defined.')
except:
    print('parameters have been defined.')
    
print("learn Rate =",FLAGS.lr, "keep Prob=", FLAGS.keep_prob, " image augument=",FLAGS.augument)   
CONTINUE = 0

parameters were defined.
learn Rate = 0.01 keep Prob= 0.5  image augument= True


In [2]:
ckpt_dir = 'ckpt_6/'
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)
log_dir = 'log_6/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

In [3]:
with tf.device('/cpu:0'):
    with tf.name_scope('input'):
        x = tf.placeholder(tf.float32, [None, 32,32,3], name='x_input')
        keep_prob = tf.placeholder(tf.float32, name='keep_prob')
        y_ = tf.placeholder(tf.int64, [None], name='labels')

In [4]:
with tf.device('/cpu:1'):
    with tf.name_scope('prediction'):
        le_net5 = LeNet5_LERU(x, keep_prob)
        y = le_net5.prediction

conv_1:  (?, 28, 28, 12)
pool_1:  (?, 14, 14, 12)
conv_2:  (?, 10, 10, 24)
pool_2:  (?, 5, 5, 24)
flat_1: (?, 600)
fc_1:  (?, 120)
fc_2:  (?, 84)
fc_3:  (?, 10)
drop_out:  (?, 10)
prediction:  (?, 10)


In [5]:
with tf.device('/cpu:2'):
    with tf.name_scope('cross_entropy'):
        cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,
                                                                                  labels=y_, 
                                                                                  name="cross_entropy_per_example"))

In [6]:
with tf.device('/cpu:3'):
    with tf.name_scope('train_step'):
        train_step = tf.train.AdagradOptimizer(FLAGS.lr).minimize(cross_entropy)

In [7]:
with tf.device('/cpu:0'):
    prediction =tf.argmax(y, 1)
    accuracy = tf.reduce_mean(tf.cast( tf.equal(prediction,y_), tf.float32))

In [ ]:
def SaveCheckpoint2S3(ori_dir,des_dir):
    bucketname = 'sagemaker-deechean-dl' 
    for file in os.listdir(file_dir):
        if os.path.isfile(file_dir+file):
            print('save files to s3')
            aws_boto3.upload_file(file_dir+file, bucketname,des_dir+file)

In [ ]:
import time

if __name__ == '__main__':
    data = cifar10();
    with tf.Session(config=tf.ConfigProto(
                            device_count={"CPU":4},
                            inter_op_parallelism_threads=1,
                            intra_op_parallelism_threads=1)) as sess:
        sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver(max_to_keep=1)
        if CONTINUE != 0:
            model_file=tf.train.latest_checkpoint(ckpt_dir)
            saver.restore(sess,model_file)
        for i in range(CONTINUE, FLAGS.epoch):
            train_image, train_label,_ = data.get_train_batch(FLAGS.batch_size)
            loss, _,accuracy_rate = sess.run([cross_entropy, train_step,accuracy], feed_dict={keep_prob: FLAGS.keep_prob, x:train_image, y_:train_label})
            tfg.savelog(log_dir,'train_accuracy',i, accuracy_rate)  
            tfg.savelog(log_dir,'train_loss',i, loss)   
            if (i+1) % FLAGS.ckpt_frequency == 0:  #保存预测模型
                saver.save(sess,ckpt_dir+'cifar10_'+str(i+1)+'.ckpt',global_step=i+1)
                #SaveCheckpoint2S3()
                acc_accuracy = 0
                for j in range(int(10000/FLAGS.test_size)):                    
                    test_image, test_label,test_index = data.get_test_batch(FLAGS.test_size)
                    accuracy_rate, output = sess.run([accuracy,prediction],feed_dict={keep_prob: 1, x:test_image, y_:test_label})
                    acc_accuracy += accuracy_rate
                    tfg.savelog(log_dir,'test_index',i, test_index)
                    tfg.savelog(log_dir,'output',i, output)
                accuracy_rate = acc_accuracy/10000*FLAGS.test_size
                print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + ' iter ' + str(i) + ', Test accuracy:' +str(round(accuracy_rate*100,2))+'%')
                tfg.savelog(log_dir,'test_accuracy',i, accuracy_rate)                  
    tf.reset_default_graph()

train set length: 50000
test set length: 10000
2019-06-09 14:00:50 iter 249, Test accuracy:12.04%
2019-06-09 14:07:48 iter 499, Test accuracy:13.12%
2019-06-09 14:14:12 iter 749, Test accuracy:12.61%
2019-06-09 14:20:34 iter 999, Test accuracy:14.85%
2019-06-09 14:26:57 iter 1249, Test accuracy:20.28%
2019-06-09 14:32:57 iter 1499, Test accuracy:20.71%
2019-06-09 14:40:57 iter 1749, Test accuracy:21.76%
2019-06-09 14:55:43 iter 1999, Test accuracy:22.44%
2019-06-09 15:10:39 iter 2249, Test accuracy:22.56%
2019-06-09 15:25:41 iter 2499, Test accuracy:23.12%
2019-06-09 15:40:42 iter 2749, Test accuracy:23.69%
2019-06-09 15:57:29 iter 2999, Test accuracy:23.92%
2019-06-09 16:13:54 iter 3249, Test accuracy:24.53%
2019-06-09 16:28:16 iter 3499, Test accuracy:24.88%
2019-06-09 16:41:20 iter 3749, Test accuracy:24.68%
2019-06-09 16:57:13 iter 3999, Test accuracy:26.04%
2019-06-09 17:11:27 iter 4249, Test accuracy:26.47%
2019-06-09 17:26:36 iter 4499, Test accuracy:26.46%
2019-06-09 17:40:47 i